# Autograd를 사용한 자동미분
신경망 학습시 자주 사용되는 알고리즘은 *역전파*
매개변수는 주어진 매개변수에 대한 손실함수의 변화도에 따라 조정됨.

`torch.autograd`는 자동 미분 엔진. 계산 그래프에 대한 변화도의 자동 계산을 지원

In [2]:
import torch

x = torch.ones(5) # 입력텐서: 값이 모두 1인 1차원 텐서
y = torch.zeros(3) # 목표 텐서: 값이 모두 0인 1차원 텐서
w = torch.randn(5, 3, requires_grad=True) # 크기 (5, 3)
# requires_grad=True: 이 텐서는 학습 가능하며 역전파동안 그래디언트가 계산된다.
b = torch.randn(3, requires_grad=True) # 크기 3
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)


`requires_grad`의 값은 텐서를 생성할 때 설정하거나, `x.requires_grad(True)`를 사용해 설정할 수 있음
연산그래프를 위해 텐서에 적용하는 함수는 `Function`의 클래스의 객체. 

1. 순전파 방향으로 함수를 계산하는 방법
2. 역방향 전파 단계에서 도함수(derivative)를 계산하는 방법도 알고 있음.
- 역방향 전파 함수에 대한 참조(reference)는 텐서의 `grad_fn` 속성에 저장된다.

In [3]:
print(f'Gradient function for z={z.grad_fn}')
print(f'Gradient function for loss={loss.grad_fn}')


Gradient function for z=<AddBackward0 object at 0x7f60204e0f40>
Gradient function for loss=<BinaryCrossEntropyWithLogitsBackward0 object at 0x7f60204e1390>


변화도(Gradient) 계산하기
매개변수의 가중치를 최적화하려면 손실함수의 도함수를 계산해야 한다. 
x와 y의 일부 고정값에서 loss값을 가져와야 한다.
`loss.backward()`를 호출한 다음, `w.grad`와 `b.grad`에서 값을 가져옴

In [4]:
loss.backward()
print(w.grad)
print(b.grad)


tensor([[0.2164, 0.0005, 0.0922],
        [0.2164, 0.0005, 0.0922],
        [0.2164, 0.0005, 0.0922],
        [0.2164, 0.0005, 0.0922],
        [0.2164, 0.0005, 0.0922]])
tensor([0.2164, 0.0005, 0.0922])


`requires_grad`가 True로 설정된 노드의 grad만 구할 수 있다. 다른 노드에서는 유효하지 않음

성능 때문에 `backward`를 사용한 gradient 계산은 한번만 수행할 수 있다.

여러번 계산할 시엔 `backward`호출 시 `retrain_graph=True`를 전달해야 함

`requires_grad=True`인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원함. 

모델을 학습한 뒤 입력데이터를 적용하는 순전파 연산만 필요한 경우엔 추적이 필요 없음.

`torch.no_grad()`블록으로 감싸서 추적을 멈출 수 있음


In [5]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
	z = torch.matmul(x, w)+b
print(z.requires_grad)


True
False


In [6]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)


False


변화도 추적을 멈춰야 하는 이유
- 신경망의 일부 매개변수를 고정으로 표시(frozen parameter)
- 변화도를 추적하지 않는 텐서의 연산이 더 효율적, 순전파 단계만 수행할 때 연산속도 향상

- DAG(Directed Acyclic Graph)
- `autograd`는 데이터(텐서)의 실행된 모든 연산들의 기록을 `Function`객체로 구성된 방향성 비순환 그래프(DAG)에 저장함
- DAG의 leaf는 입력 텐서, root는 결과텐서

###  forward에서 autograd가 하는 역할
- 요청된 연산을 수행해 결과 텐서를 계산
- DAG에 연산의 변화도 기능(gradient function)을 유지

### backward에서 autograd가 하는 역할
- DAG의 root에서 `.backward()`가 호출될 때 시작함. 
- 각 `.grad_fn`으로부터 변화도를 계산
- 각 텐서의 `.grad` 속성에 계산 결과를 쌓고(accumulate)
- 연쇄 법칙을 사용해, 모든 leaf 텐서들까지 전파함(propagate)


# 선택적 읽기(Optional Reading): 텐서 변화도와 야코비안 곱(Jacobian Product)
대부분은 스칼라 손실 함수를 가지고 일부 매개변수와 관련한 변화도를 계산해야 함. 그러나 출력 함수가 임의의 텐서인 경우, PyTorch는 실제 변화도가 아닌 Jacobian Product를 계산해야 함

- 야코비안 행렬 자체를 계산하는 대신, 주어진 입력 벡터 $v = (v_1...v_m)$에 대한 $Jacobian Productv^T \dot J$를 계산한다. 
- v를 인자로 `backward`를 호출하면 이뤄짐. v의 크기는 product를 계산하려고 하는 원래 텐서의 크기와 같아야 함.

In [10]:
inp = torch.eye(4, 5, requires_grad=True)
print(f'normal call\n{inp.grad}')
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f'First call\n{inp.grad}')

out.backward(torch.ones_like(out), retain_graph=True)
print(f'Second call\n{inp.grad}')

inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f'Call after zeroing gradients\n{inp.grad}')


normal call
None
First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])
Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])
Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


- `backward`를 두차례 호출하면 값이 달라짐. 변화도를 누적하기 떄문.
- 계산된 Gradient의 모든 값이 leaf노드의 `grad`속성에 추가됨. 
- 제대로 계산하기 위해서는 `grad` 속성을 0으로 만들어야 하는데 `optimizer`가 이 과정을 도와줌